### Step 1 - Start Spark Session, include additional configurations and common functions, and read data from the spark-warehouse database

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [ ]:
appName = "Constructors Ingestion"
spark = create_spark_context(appName, warehouse_location)

In [ ]:
constructor_df = spark.read.parquet(f"{warehouse_location}/f1_raw.db/constructors")

### Step 2 - Rename and drop columns, and add new columns

In [ ]:
constructor_column_mapping = {
    "constructorId": "constructor_id",
    "constructorRef": "constructor_ref"
}

constructor_df = rename_columns(constructor_df, constructor_column_mapping)


In [ ]:
columns_to_drop = ['url']

constructor_df = drop_columns(constructor_df, columns_to_drop)

In [ ]:
constructor_df = add_ingestion_date(constructor_df)

### Step 3 - Write a data to a Datalake as a Parquet

In [ ]:
constructor_df.write.mode("overwrite").parquet(f"{processed_folder_path}/constructors")